In [ ]:
每日主題知識點
● 實作線性迴歸模型

簡介
● 我們將在本日以程式實作預測糖尿病嚴重程度，採用由 Efron et al. (2004) 發表之人體健康模擬資料，在下一日的課程會對程式碼做數學上的分析，敬請期待。

程式實作
● 首先要導入需用套件與資料集，讓程式知道我們要準備拿這筆資料來做訓練了。



1
import numpy as np
2
import matplotlib.pyplot as plt
3
from IPython.display import set_matplotlib_formats
4
from sklearn.datasets import load_diabetes


● 再來我們整理待訓練資料



1
diabetes = load_diabetes()
2
data, yt = diabetes.data, diabetes.target  # data 每一筆資料為一個病人 yt為對應到data的疾病狀態
3
feature_names = diabetes.feature_names
4
print('輸入資料', data.shape, yt.shape)
5
print('資料的特徵名', feature_names)
6
​
7
# 單看 age（年齡） 這筆特徵
8
x_data = data[:,feature_names == 'age']
9
print('整理後', x_data.shape)
輸出如下:

輸入資料 (442, 10) (442,)
特徵包含：["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"]

整理後 (442, 1)



● 再來我們要再 x_data 中每一筆資料前面再加上數值1(虛擬變數(下一日會介紹))，就完成準備資料了。



1
# 在 x 向量第 1 個位置中加入虛擬變數
2
x = np.insert(x_data, 0, 1.0, axis=1)
3
print('加入虛擬變數後', x.shape)
4
​
5
# 整理完後的資料長相
6
print(x.shape)
7
​
8
# 看看前五間房的房間數(加上虛擬變數)如何對應真實房價
9
print(x[:5,:])
10
print(yt[:5])
輸出如下:

加入虛擬變數後 (442, 2)
(442, 2)
[[1.    0.05068012]
[1.    -0.04464164]
[1.    0.05068012]
[1.    -0.04464164]
[1.    -0.04464164]]

[24.  21.6 34.7 33.4 36.2]



● 我們在此可以把房間數與房價的關係畫成散佈圖



1
# 畫出資料散佈圖
2
plt.scatter(x[:,1], yt, s=10, c='b')  # 因為 x 每筆資料的第一個位置是虛擬變數，我們只要取第二個位置的資料來畫
3
plt.xlabel('age', fontsize=14)
4
plt.ylabel('diabetes progress', fontsize=14)
5
plt.show()


● 還記得我們要找出一個函數，用來預測新的輸入資料嗎，他稱為預測函數，預測函數可簡單也可複雜，再給予一個預測函數後，及訓練出最佳參數，該預測函數就是我們最後的模型，所以在此我們要先給一個預測函數，那我們就給一個簡單的吧。



1
# 以預測函數 (1, x) 之值計算預測值 yp
2
def pred(x, w):
3
    return(x @ w)  # @ 是兩向量做內積的意思


● 接下來初始化我們的各個超參數



1
# 資料樣本總數
2
M  = x.shape[0]
3
​
4
# 輸入資料之維數（含虛擬變數）
5
D = x.shape[1]
6
​
7
# 迭代運算次數 (可調整)
8
iters = 50000
9
​
10
# 學習率 (可調整)
11
alpha = 0.01
12
​
13
# 權重向量的初始值（預設全部為 1，可調整），我們主要目的就是找出此變數的最佳值!
14
w = np.ones(D)
15
​
16
# 記錄評估結果用（僅記錄損失函數值）
17
history = np.zeros((0,2))


● 主程式 (梯度下降法的實作)



1
for k in range(iters):
2
    
3
    # 計算預測值
4
    yp = pred(x, w)
5
    
6
    # 計算誤差
7
    yd = yp - yt
8
    
9
    # 梯度下降法的實作
10
    w = w - alpha * (x.T @ yd) / M
11
    
12
    # 繪製學習曲線所需資料之計算與儲存
13
    if ( k % 100 == 0):
14
        # 計算損失函數值
15
        loss = np.mean(yd ** 2) / 2
16
        # 記錄計算結果
17
        history = np.vstack((history, np.array([k, loss])))
18
#         如果覺得沒感覺，可以跑下面那行
19
#         print( "迭代第 %d 次  loss 為 %f" % (k, loss))


● 損失函數值的改變



1
print('損失函數初始值: %f' % history[0,1])
2
print('損失函數最終值: %f' % history[-1,1])
輸出如下:

損失函數初始值: 154.224934
損失函數最終值: 21.800276



● 前面已找出最佳的w，完成訓練階段，接下來進入我們的預測階段，我們要讓此模型預測房間數嘴少與最多的房價，再藉由這兩筆資料與預測出數值畫成一條線性回歸線(兩點連成一線)，並繪製於剛剛的散佈圖上



1
# 繪製散佈圖與迴歸線
2
​
3
# 提取每一筆病人年齡資訊
4
xall = x[:,1].ravel()
5
​
6
# 因為要丟回原本的預測函數，所以要加上虛擬變數，且取兩點畫圖即可(要畫的是直線)
7
xl = np.array([[1,xall.min()],[1,xall.max()]])
8
yl = pred(xl,w)  # 此時的 w 已訓練完畢
9
​
10
plt.figure(figsize=(6,6))
11
plt.scatter(x[:,1], yt, s=10, c='b')
12
plt.xlabel('age', fontsize=14)
13
plt.ylabel('diabetes progress', fontsize=14)
14
plt.plot(xl[:,1], yl, c='k')
15
plt.show()




● 繪製學習曲線，可以從這條曲線看出收斂(訓練完成)的速度



1
# 繪製學習曲線（第一組數除外）!注意!若前面的主程式有多跑，這邊的圖形會不一樣，因為history被多儲存了，正確的應該如下
2
plt.plot(history[1:,0], history[1:,1])
3
plt.xlabel('iter')
4
plt.ylabel('loss')
5
plt.show()
知識點總結
● 實作線性回歸模型



實作步驟簡單歸納:



導入資料 → 整理資料 → 建立預測函數 → 初始化超參數 → 主程式



無論多麼撲朔迷離的問題基本上大同小異，但在某些步驟會加入不同的演算法做適當的調整，所以清楚掌握這些基本步驟的原理及數學方法是最必須要會的，若想進一步加強，也務必多了解不同的演算法。

延伸閱讀
● 今天我們只取一個特徵值做訓練，底下分享一篇文章，作者使用全部特徵值(如同我們剛剛看到的13筆)做訓練:

https://ithelp.ithome.com.tw/articles/10235449